In [4]:
from sansdata import *

## 0. Compute relative pixel efficiencies from perspex measurement

In [ ]:
sans_file = SansData(f"data/perspex.mpa")

In [ ]:
# Helper function for indexing a rectangle from an array
def rect_slice(center, size):
    x, y = center
    return (slice(y - size // 2, y + size // 2), slice(x - size // 2, x + size // 2))

N_eff_pixel = 550  # just an estimate for now
beamstop_x, beamstop_y = 520, 1024 - 500
beamstop_w, beamstop_h = 76, 76
flood_intensities_full = sans_file.raw_intensity.astype(float)
flood_intensities_full[rect_slice((beamstop_x, beamstop_y),beamstop_w)] = 0

# flood_intensities_full[beamstop_y-beamstop_h//2:beamstop_y+beamstop_h//2,beamstop_x-beamstop_w//2:beamstop_x+beamstop_w//2] = 0
flood_intensities = flood_intensities_full

# Compensate for the fact that the beamstop square is not considered in the average
area_factor = 1/(1- beamstop_w * beamstop_h/N_eff_pixel ** 2)
print(area_factor)
mu_flood = np.mean(flood_intensities[rect_slice((512, 512), N_eff_pixel)]) * area_factor
flood_norm = flood_intensities / mu_flood

def plot_image(data, label):
    plt.figure()
    extent = [550//2, 1024-550//2, 550//2, 1024-550//2]
    plt.imshow(
        data, cmap="viridis", extent=extent, aspect="auto"
    ) 
    plt.colorbar(label=label)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()
plot_image(flood_norm, "Relative efficiency")

### 0.1 Synthesize full efficiency map (temporary)
Currently, no flood measurement is available that can characterize the full detector so make up the values in the beamstop area for now by copying other values. NOT FOR USE IN ACTUAL DATA REDUCTIONS.

In [ ]:
flood_norm_synth = np.copy(flood_norm)
offset = 15
flood_norm_synth[rect_slice((beamstop_x, beamstop_y),beamstop_w+offset)] = flood_norm_synth[rect_slice((beamstop_x, beamstop_y+beamstop_w+offset+10),beamstop_w+offset)]
plot_image(flood_norm_synth, "Relative efficiency")